## Ingestão Contínua de dados em Data Lake com Glue e visualização em dashboard no QuickSight   

### Pré-requisitos   
Antes de iniciar, é importante ressaltar a necessidade de criar uma conta na AWS. A amazon oferece um [plano gratuito](https://aws.amazon.com/pt/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsm.page-all-free-tier=5), onde você terá acesso a todos serviços que iremos utilizar durante o projeto sem pagar nada. Além disso, podemos interagir com a plataforma da AWS de diferentes formas, por meio do [Console Web](https://aws.amazon.com/pt/console/), via [AWS Cli](https://aws.amazon.com/pt/cli/) ou via [API](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).  

Com sua conta em mãos, vamos em frente ao infinito e além!🚀

# Arquitetura
![](img/arquitetura.jpg)

## Base de dados : [COVID19-BR](https://github.com/wcota/covid19br)  
Iremos utilizar o dataset mantido pelo [Wesley Cota](https://wesleycota.com) com atualizações diárias de números relacionados à COVID-19 no Brasil. Há dados referentes aos suspeitos e vacinados (primeira e segunda dose), novos casos e mortes confirmadas, recuperados e testes realizados. Para descrição completa dos dados, acesse [Descrição dos dados e utilização](https://github.com/wcota/covid19br/blob/master/DESCRIPTION.md).

* ### cases-brazil-total.csv: Dados nacionais de COVID-19;

In [5]:
import pandas as pd

In [6]:
cases_brazil_total = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-total.csv")

In [79]:
cases_brazil_total.columns

Index(['country', 'state', 'totalCases', 'totalCasesMS', 'notConfirmedByMS',
       'deaths', 'deathsMS', 'URL', 'deaths_per_100k_inhabitants',
       'totalCases_per_100k_inhabitants', 'deaths_by_totalCases', 'recovered',
       'suspects', 'tests', 'tests_per_100k_inhabitants', 'vaccinated',
       'vaccinated_per_100k_inhabitants', 'vaccinated_second',
       'vaccinated_second_per_100k_inhabitants', 'date', 'newCases',
       'newDeaths'],
      dtype='object')

In [80]:
cases_brazil_total.head()

,country,state,totalCases,totalCasesMS,notConfirmedByMS,deaths,deathsMS,URL,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,...,suspects,tests,tests_per_100k_inhabitants,vaccinated,vaccinated_per_100k_inhabitants,vaccinated_second,vaccinated_second_per_100k_inhabitants,date,newCases,newDeaths
0,Brazil,TOTAL,12577643,12573615,4028,314059,313866,https://covid.saude.gov.br/,148.31195,5939.69535,...,5430111,41904799,19789.21964,16235182,7666.94007,4812503,2272.66760,2021-03-29,35429,1615
1,Brazil,AC,68905,68905,0,1240,1240,http://saude.acre.gov.br/,140.59993,7812.93406,...,42505,181661,20598.00325,46155,5233.37888,11752,1332.52451,2021-03-29,163,6
2,Brazil,AL,152210,152210,0,3511,3511,http://cidadao.saude.al.gov.br/,105.20301,4560.79466,...,10071,368601,11044.69795,224377,6723.19443,56243,1685.25573,2021-03-29,386,22
3,Brazil,AM,346320,346320,0,11974,11974,http://www.amazonas.am.gov.br/,288.90626,8355.93907,...,425,852071,20558.59713,416051,10038.39457,125340,3024.17823,2021-03-29,510,22
4,Brazil,AP,96199,96199,0,1279,1279,https://saude.portal.ap.gov.br/,151.23012,11374.65695,...,2990,166052,19634.13899,49368,5837.31707,16235,1919.64112,2021-03-29,550,11


 * ### cases-brazil-cities.csv: Dados municipais de COVID-19;  

In [81]:
cases_brazil_cities = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv")

In [82]:
cases_brazil_cities.columns

Index(['country', 'state', 'city', 'ibgeID', 'cod_RegiaoDeSaude',
       'name_RegiaoDeSaude', 'deaths', 'totalCases',
       'deaths_per_100k_inhabitants', 'totalCases_per_100k_inhabitants',
       'deaths_by_totalCases', '_source', 'date', 'newCases', 'newDeaths',
       'last_info_date'],
      dtype='object')

In [83]:
cases_brazil_cities.head()

,country,state,city,ibgeID,cod_RegiaoDeSaude,name_RegiaoDeSaude,deaths,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,date,newCases,newDeaths,last_info_date
0,Brazil,GO,Abadia de Goiás/GO,5200050,52001.0,Central,18,1124,200.93771,12547.44363,0.01601,MS,2021-03-29,-1,0,2021-03-29
1,Brazil,MG,Abadia dos Dourados/MG,3100104,31074.0,Patrocínio / Monte Carmelo,11,324,157.00828,4624.60748,0.03395,MS,2021-03-29,0,0,2021-03-29
2,Brazil,GO,Abadiânia/GO,5200100,52011.0,Pirineus,20,298,97.74693,1456.42930,0.06711,MS,2021-03-29,0,0,2021-03-29
3,Brazil,PA,Abaetetuba/PA,1500107,15011.0,Tocantins,134,6077,84.23435,3820.09052,0.02205,MS,2021-03-29,1,1,2021-03-29
4,Brazil,MG,Abaeté/MG,3100203,31024.0,Sete Lagoas,16,731,68.81720,3144.08602,0.02189,MS,2021-03-29,0,0,2021-03-29


* ### gps_cities.csv: Coordenadas GPS de cada município.

In [86]:
gps_cities = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/gps_cities.csv")

In [87]:
gps_cities.columns

Index(['ibgeID', 'id', 'lat', 'lon', 'longName'], dtype='object')

In [88]:
gps_cities.head()

,ibgeID,id,lat,lon,longName
0,2600054.0,Abreu e Lima/PE,-7.900719,-34.898389,"Abreu e Lima, Região Geográgica Imediata do Re..."
1,2600104.0,Afogados da Ingazeira/PE,-7.750429,-37.635638,"Afogados da Ingazeira, Microrregião de Pajeú, ..."
2,2600203.0,Afrânio/PE,-8.519892,-41.007020,"Afrânio, Região Geográfica Imediata de Petroli..."
3,2600302.0,Agrestina/PE,-8.455802,-35.944266,"Agrestina, Região Geográfica Imediata de Carua..."
4,2600401.0,Água Preta/PE,-8.707911,-35.519859,"Água Preta, Microrregião da Mata Meridional Pe..."


## 1. Criação e Ingestão Contínua de dados em Data Lake

### Amazon S3 *(Simple Storage Service)*  
O [Amazon S3](https://aws.amazon.com/pt/s3/) é o serviço de armazenamento mais utiliado para Data Lakes em todo mundo oferecendo escalabilidade, segurança e performance. O Amazon S3 armazena dados como objetos dentro de **buckets**. É possível controlar permissões de acesso para cada bucket e usuários.

 Podemos criar um bucket fácilmente via terminal através do AWS CLI:
 ```
 aws s3 mb s3://bucket-name
 ```  
 
 
  Lembrando que o nome do bucket deve: 
 - ser exlusivo em toda AWS;
 - conter caracteres minúsculos, números, pontos e traços

In [91]:
!aws s3 mb s3://datalake-projeto-covid

make_bucket: datalake-projeto-covid


Nossos arquivos serão adicionados automaticamente através da função Lambda que iremos criar. No entanto, o [arquivo](https://github.com/wcota/covid19br/blob/master/gps_cities.csv) com as coordenadas GPS dos município é estático, portanto podemos realizar o [download](https://github.com/wcota/covid19br/blob/master/gps_cities.csv) em nossa máquina local e mover diretamente para o nosso bucket recém-criado: 

 ```
aws s3 mv filename s3://bucket-name
 ```

In [95]:
!aws s3 mv gps_cities.csv s3://datalake-projeto-covid

Completed 256.0 KiB/961.3 KiB (116.6 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/961.3 KiB (163.6 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/961.3 KiB (190.3 KiB/s) with 1 file(s) remaining
Completed 961.3 KiB/961.3 KiB (191.7 KiB/s) with 1 file(s) remaining
move: .\gps_cities.csv to s3://datalake-projeto-covid/gps_cities.csv


Para listar os arquivos em um bucket específico:  
```
aws s3 ls s3://bucket-name
```

In [97]:
!aws s3 ls s3://datalake-projeto-covid

2021-03-29 21:46:39     984398 gps_cities.csv


### AWS Lambda  
[Lambda](https://aws.amazon.com/pt/lambda/) é um serviço da AWS que nos permite executar funções sem provisionar ou gerenciar servidores, precisamos apenas fornecer um arquivo .zip contendo nossa função e as dependências necessárias.  

Para criar uma função Lambda:
1. Acesse *Funções* em seu serviço *AWS Lambda* no [Console Web](https://aws.amazon.com/pt/console/) e clique em *Criar Função*.  
2. Defina as informações básicas como nome da sua função e qual linguagem de programação será utilizada e clique em *Criar Função*.  

Antes de prosseguir, precisamos entender alguns conceitos referente ao serviço AWS Lambda, abaixo temos uma visão geral de nossa função recém-criada.
![](img/lambda/visao_geral.jpg)  

**Gatilho ou Trigger:** Podemos definir um gatilho para que nossa função seja executada sempre que um evento específico ocorrer. Iremos definir posteriormente um gatilho CloudWatch Event para que nossa função seja executada diariamente em um horário específico.  

**Layers**: Layers são nossas dependências empacotadas, arquivos .zip contendo as bibliotecas que serão utilizadas pela nossa função. As layers criadas podem ser compartilhadas entre as demais funções lambdas.(https://docs.aws.amazon.com/pt_br/lambda/latest/dg/configuration-layers.html).  

**Destinos**: Podemos enviar registros de invocação a um destino quando a função for executada. Ex: Enviar um e-email para o desenvolvedor responsável pela função em caso de falha na execução.


Estrutura da função lambda:  
![](img/lambda/event_context.jpg)  

**Manipulador**: O manipulador da função Lambda é o método no código que processa eventos. Quando a função é invocada, o método é executado, por padrão o nome do método é ***lambda_handler***. E o nome do manipulador é derivado do nome_arquivo.nome_método, portanto por padrão o nome do manipulador é **lambda_function.lambda_handler**. Caso desejar um nome diferente para seu arquivo ou método da função, é preciso atualizar o nome do manipulador em *Configurações de tempo de execução (Runtime Settings)*.  
![](img/lambda/manipulador.jpg)  

Quando a função Lambda é invocada, o método recebe dois argumentos, **event** e **context**:  
* **event**: Um evento é um documento JSON que contém os dados a serem processados pela função. Lembra que podemos definir um evento gatilho para nossa função? Então, o argumento event recebe as informações desse evento. Ex: Podemos definir um evento de PutObject no S3 como gatilho, ou seja, sempre que um novo arquivo for adicionado ao bucket especificado, nossa função será invocada. E através do argumento **event** podemos acessar informações como o nome do arquivo adicionado por exemplo.  

* **context**: O contexto é um objeto contendo informações referentes a invocação da função, como horário, ambiente de executação, entre outros.  

Compreendido alguns conceitos essenciais, podemos escrever nossa função Lambda que será responsável por:  
* Coletar as informações diárias da COVID19 disponíveis nos arquivos cases-brazil-cities.csv e cases-brazil-total.csv no repositório [covid19br](https://github.com/wcota/covid19br).  
* Converter para o formato Apache Parquet, formato mais otimizado e performático em relação ao CSV. Para mais informações acesse [Apache Parquet vs. CSV Files](https://dzone.com/articles/how-to-be-a-hero-with-powerful-parquet-google-and).  
* Por fim, enviar os arquivos diário para o nosso Data Lake em uma folder específica para cada dia.  

Substitua o código em lambda_function.py pelo abaixo e clique em ***Deploy***:


```
import pandas as pd
import awswrangler as wr

def get_data(bucket, data):
    try:
        url = f"https://raw.githubusercontent.com/wcota/covid19br/master/{data}.csv"
        df = pd.read_csv(url)
        date = df['date'][0]
        wr.s3.to_parquet(df=df, path=f's3://{bucket}/{data}/{date}/{data}.parquet')
    except Exception as exc:
        raise exc
        
def lambda_handler(event, context):
    get_data('datalake-projeto-covid', 'cases-brazil-total')
    get_data('datalake-projeto-covid', 'cases-brazil-cities')
```

![](img/lambda/lambda-deploy.jpg)

Temos duas dependências em nossa função, [awswrangler](https://pypi.org/project/awswrangler/) e [pandas](https://pandas.pydata.org). 
Como explicado anteriormente, layers são pacotes .zip contendo as dependências necessárias da nossa função. A biblioteca awswrangler já incluí o pandas, portanto iremos empacotar apenas ela.

1. Primeiramente, realize o download da awswrangler para Python 3.8 em [Awswrangler Layer](https://github.com/awslabs/aws-data-wrangler/releases/download/2.6.0/awswrangler-layer-2.6.0-py3.8.zip);  
2. Faça o upload do arquivo .zip em um bucket no S3, é a recomendação para arquivos maiores do que 10 MB. 
    ```
    aws s3 mv awswrangler-layer-2.6.0-py3.8.zip s3://your-bucket-name/
    ```
3. Acesse o menu do serviço AWS Lambda, a seguir ***Camadas (Layers)*** e ***Criar uma camada***.
4. Especifique o nome da sua função, localização do arquivo .zip e selecione Python 3.8 em *Tempos de Execução* e por fim, ***Criar***.  
![](img/lambda/create-layer.jpg)

5. Com a Layer criada, precisamos adicioná-la na função. Para isso, acesse a função lambda criada, menu ***Camadas*** e ***Adicionar uma Camada***:
![](img/lambda/adicionar-camada.jpg)

6. Escolha a opção ***Camadas personalizadas*** e selecione a layer criada:  
![](img/lambda/adicionar-camada-2.jpg)



Por padrão, é alocado 128 MB de memória para execução da função. Portanto, nossa função irá utilizar um pouco mais, acesse o menu ***Configuração > Configuração Geral > Editar*** e define 250 MB de memória:  
![](img/lambda/config-memoria.jpg)  

Todas configurações realizadas, podemos invocar nossa função manualmente através de um evento de teste:  
![](img/lambda/teste-funcao.jpg)

Agora basta irmos em nosso Data Lake e verificar que os dados foram coletados e adicionados:  
![](img/lambda/dados-datalake.jpg)

Para finalizar essa primeira etapa do projeto, queremos que nossa função seja invocada automaticamente, para isso precisamos definir um gatilho. Na visão geral da função, clique em ***Adicionar Gatilho***, nosso gatilho será do tipo ***EventBridge (CloudWatch Events)*** com execução diária programada para 22 horas da noite. É possível definir diferentes configurações de execução através da expressão cron, como minutos, dias da semana, dias do mês, mês. Saiba mais em [Schedule Expressions for Rules](https://docs.aws.amazon.com/AmazonCloudWatch/latest/events/ScheduledEvents.html).  
![](img/lambda/gatilho.jpg)  

![](img/lambda/gatilho-criado.jpg)

## 2. Crawler e análise dos dados utilizando SQL   
O AWS Glue é um serviço de ETL *(Extract, Transform and Load)* auto gerenciado, sem a necessidade de provisiona infraestrutura. Iremos utilizá-lo para conectar ao nosso bucket no S3, reconhecer os dados e catalogar em tabelas no Glue Data Catalog, disponibilizando para consultas SQL no Athena e criação do dashboard no QuickSight.  

* ### Adicionar Banco de Dados

* ### Criar Crawler  
 [Documentação](https://docs.aws.amazon.com/pt_br/glue/latest/dg/what-is-glue.html)

* ### AWS Athena

## 3. Criação de dashboard para visualização dos dados  
* ## Amazon QuickSight
